In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import yfinance as yf
import pickle

In [2]:
np.random.seed(0)

In [3]:
# load and group by cap size
stocks_screener = pd.read_csv('nasdaq_screener_1637428146284.csv') #https://www.nasdaq.com/market-activity/stocks/screener
stocks_screener = stocks_screener[stocks_screener['Market Cap'].notna()]

M = 10**6
B = 10**9

bins =  np.array([0,50*M,300*M,2*B,10*B,200*B,np.inf])
ind = np.digitize(stocks_screener['Market Cap'], bins)

stocks_screener['Cap Group'] = ind

gb = stocks_screener.groupby('Cap Group')
list_of_groupbed_stocks = [gb.get_group(x) for x in gb.groups]

cap_list = ['nano','micro','small','medium','large','mega']
stock_cap_dict = dict()
for i in range(len(cap_list)):
    stock_cap_dict[cap_list[i]] = list_of_groupbed_stocks[i]['Symbol']
    


In [4]:
for cap in cap_list:
    print(cap + ' ' + str(len(stock_cap_dict[cap])))

nano 1216
micro 880
small 1077
medium 503
large 225
mega 19


In [5]:
def pick_stock_get_data(stock_dict = stock_cap_dict,cap = 'small',num_stocks = 100,want_log = True,start_date="2018-01-01",
                        end_date="2021-12-31",cut_date = '2021-01-01' ):
    
    #length standards
    stock_data = yf.download('AAPL', start=start_date, end=end_date,progress=False).reset_index()
    standard_length = len(stock_data)
    
    count = 0
    
    resample_count = 0
    
    train_data = {}
    test_data = {}
    
    while count<num_stocks:
        
        length = 0
        
        resample_count-=1
        
        while length!=standard_length:
            resample_count+=1
            stock_sample =  list((stock_dict[cap]).sample(n=1))[0]
            stock_data = stock_data.replace(0, np.nan)
            stock_data = yf.download(stock_sample, start=start_date, end=end_date,progress = False).reset_index().dropna()
            
            length = len(stock_data)
        
        
        stock_data['daily_return'] = (stock_data.Close /stock_data.Close.shift(-1)).dropna()
        
        stock_data['Log_daily_return'] = np.log(stock_data.Close /stock_data.Close.shift(-1).dropna())
        
        train_full, test_full = stock_data[stock_data.Date<cut_date],stock_data[stock_data.Date>=cut_date]
        
        if want_log:
        
            train_data[stock_sample],test_data[stock_sample] = np.array(train_full[['Date','Log_daily_return']]),np.array(test_full[['Date','Log_daily_return']].dropna())
            
        else:
            
            train_data[stock_sample],test_data[stock_sample] = np.array(train_full[['Date','daily_return']]),np.array(test_full[['Date','daily_return']].dropna())
        
        count+=1
        
    return train_data,test_data

In [8]:
# getting data for hmm and garch model (with log return)
metrics_df_concat = pd.DataFrame()

for cap_size in cap_list:
    result_metric_dict = {}
    train_data, test_data = pick_stock_get_data(stock_dict = stock_cap_dict,cap = cap_size,num_stocks = 100,want_log = True,start_date="2018-01-01",
                        end_date="2021-12-31",cut_date = '2021-01-01' )
    #write train
    a_file = open('processed_data/'+cap_size+"_train.pkl", "wb")
    pickle.dump(train_data, a_file)
    a_file.close()
    
    #write test
    a_file = open('processed_data/'+cap_size+"_test.pkl", "wb")
    pickle.dump(test_data, a_file)
    a_file.close()
#     metrics_df = pd.DataFrame(result_metric_dict).T
#     metrics_df['Cap_size'] = cap_size
    
#     metrics_df_concat = metrics_df_concat.append(metrics_df)

In [10]:
# getting data for arima model (without log return)
metrics_df_concat = pd.DataFrame()

for cap_size in cap_list:
    result_metric_dict = {}
    train_data, test_data = pick_stock_get_data(
            stock_dict = stock_cap_dict,cap = cap_size,num_stocks = 100,
            want_log = False,start_date="2018-01-01",
            end_date="2021-12-31",cut_date = '2021-01-01' )
    #write train
    a_file = open('processed_data_arima/'+cap_size+"_train.pkl", "wb")
    pickle.dump(train_data, a_file)
    a_file.close()
    
    #write test
    a_file = open('processed_data_arima/'+cap_size+"_test.pkl", "wb")
    pickle.dump(test_data, a_file)
    a_file.close()
#     metrics_df = pd.DataFrame(result_metric_dict).T
#     metrics_df['Cap_size'] = cap_size
    
#     metrics_df_concat = metrics_df_concat.append(metrics_df)


1 Failed download:
- BREZR: None

1 Failed download:
- AMAOW: None

1 Failed download:
- IMAQR: None

1 Failed download:
- CLAQR: None

1 Failed download:
- IMAQR: None

1 Failed download:
- MCAER: None

1 Failed download:
- ESSCR: None


In [21]:
for cap in cap_list:
    a_file = open("processed_data/{}_train.pkl".format(cap), "rb")
    output = pickle.load(a_file)
    print(len(output), output.keys(), '\n')

81 dict_keys(['WVVIP', 'CBIO', 'CYCCP', 'CLRB', 'NSYS', 'CUEN', 'SOHO', 'PIXY', 'STRR', 'RIBT', 'ACOR', 'ALJJ', 'CYRN', 'NERV', 'SVVC', 'BKEPP', 'RBCN', 'PRPO', 'BANFP', 'DYNT', 'THMO', 'ISIG', 'DTST', 'OCC', 'ONVO', 'VLYPO', 'PALT', 'DFFN', 'ABIO', 'OXLCM', 'LIXT', 'IDRA', 'REED', 'ZIVO', 'PBCTP', 'NYMTN', 'ACER', 'PULM', 'MIND', 'CPTAG', 'PSTV', 'APDN', 'CVV', 'CNFR', 'SUMR', 'WVFC', 'YTEN', 'LMRKO', 'FITBI', 'NYMTP', 'MEDS', 'SGBX', 'AATC', 'MTBCP', 'CWBR', 'SOHOB', 'NVIV', 'AUTO', 'QUMU', 'XELB', 'GROM', 'CUBA', 'CYAN', 'BSQR', 'TANNI', 'BPOPM', 'EKSO', 'XBIO', 'CPIX', 'NURO', 'KAVL', 'NXTD', 'MARPS', 'NSEC', 'EVOL', 'TTNP', 'CRTD', 'RAND', 'IFMK', 'CHSCO', 'BBLG']) 

87 dict_keys(['NH', 'SEVN', 'OPNT', 'KRMD', 'CIZN', 'MFIN', 'REFR', 'ESCA', 'MBCN', 'FRAF', 'SYRS', 'FNWB', 'HDSN', 'OXSQ', 'MRCC', 'CLSN', 'BYFC', 'AESE', 'HSON', 'VIRC', 'CMCT', 'BCML', 'GEOS', 'VERB', 'NNBR', 'CALA', 'CTHR', 'SINO', 'EIGR', 'OTIC', 'HNRG', 'BEEM', 'PFX', 'XSPA', 'ALRN', 'LTRPA', 'AUBN', 'IDN', 'WTR

In [22]:
for cap in cap_list:
    a_file = open("processed_data_arima/{}_train.pkl".format(cap), "rb")
    output = pickle.load(a_file)
    print(len(output), output.keys(), '\n')

79 dict_keys(['SNOA', 'CYCCP', 'AATC', 'JSM', 'JCTCF', 'NTRB', 'NVFY', 'LMRKO', 'WVFC', 'ARTW', 'MIND', 'BSQR', 'POAI', 'GROM', 'OPHC', 'EFOI', 'RAVE', 'RMTI', 'DHCNI', 'TANNI', 'USEG', 'VIVE', 'VIASP', 'TANNZ', 'SCKT', 'BANFP', 'GVP', 'CLRB', 'TAYD', 'WINT', 'BBI', 'BKYI', 'WTFCM', 'ZIONP', 'LIXT', 'LPTH', 'SLRX', 'SSNT', 'JAN', 'CLRO', 'EVOL', 'FNHC', 'NURO', 'OBLG', 'SINT', 'KTRA', 'STAF', 'EAST', 'CYCC', 'CYAN', 'SGLB', 'CPIX', 'QUMU', 'FITBI', 'XBIO', 'CREX', 'XELB', 'CHSCN', 'HOVNP', 'ABIO', 'STRR', 'ONVO', 'KPRX', 'CPTAL', 'NDRA', 'DTST', 'KEQU', 'THMO', 'EVOK', 'HVBC', 'BNTC', 'MSVB', 'NYMTP', 'AUTO', 'PULM', 'CNFR', 'HTGM', 'AHPI', 'HSDT']) 

88 dict_keys(['VABK', 'AVEO', 'ADES', 'WLFC', 'GFED', 'SPPI', 'ASMB', 'MRLN', 'MLVF', 'CKPT', 'LWAY', 'MRKR', 'EVFM', 'REFR', 'NBN', 'CSSE', 'ARAV', 'CHMG', 'RVSB', 'PMTS', 'CRVS', 'MRBK', 'ESPR', 'NAII', 'MAYS', 'LTRX', 'AGRX', 'DARE', 'SNCR', 'OPOF', 'KMPH', 'LMNR', 'SEVN', 'HROW', 'SGA', 'NBEV', 'VTVT', 'NCSM', 'RNDB', 'OMEX', 'MCHX', 